# Dépenses de l'état

In [ ]:
import pandas as pd

dépense = pd.read_csv("dépense_état.csv", index_col=0)

dépense

## k-means global

On cherche le nombre de classes pour  les $k$-means

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
data = dépense

inertie = []

for nb in range(1, 10):
    kmeans = KMeans(n_clusters=nb,
                    n_init=10, 
                    max_iter=300).fit(data)
    inertie.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.lineplot(x=list(range(1, len(inertie) + 1)), 
             y=inertie, 
             legend=False,
             ax=ax)
plt.show()

5 classes à l'air d'être le point d'inflexion

In [ ]:
kmeans = KMeans(n_clusters=5,
                n_init=10, 
                max_iter=300).fit(data)

In [ ]:
kmeans.inertia_

In [ ]:
kmeans.cluster_centers_

In [ ]:
clusters_kmeans = kmeans.predict(data)

clusters_kmeans

In [ ]:
clusters = [[], [], [], [], []]

for i, cluster in enumerate(clusters_kmeans):
    clusters[cluster].append(dépense.index[i])
    
clusters

## k-means ACP

Refaite l'exercice précédent, mais uniquement sur les 2 premiers axes de l'ACP sur des données **non centrées ni réduites**.

In [ ]:
from sklearn.decomposition import PCA 
import numpy as np
import pandas

On utilise les données non centrées ni réduites :

In [ ]:
X = dépense

In [ ]:
pca = PCA()
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

In [ ]:
data = C.iloc[:, [0, 1]]

data

In [ ]:
inertie = []

for nb in range(1, 10):
    kmeans = KMeans(n_clusters=nb,
                    n_init=10, 
                    max_iter=300).fit(data)
    inertie.append(kmeans.inertia_)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.lineplot(x=list(range(1, len(inertie) + 1)), 
             y=inertie, 
             legend=False,
             ax=ax)
plt.show()

On voit que 5 classes suffisent également

In [ ]:
kmeans = KMeans(n_clusters=5,
                n_init=10, 
                max_iter=300).fit(data)

clusters_kmeans = kmeans.predict(data)

clusters = [[], [], [], [], []]

for i, cluster in enumerate(clusters_kmeans):
    clusters[cluster].append(dépense.index[i])
    
clusters

In [ ]:
#centres

centers = kmeans.cluster_centers_
centers

#couleurs 
current_palette = sns.color_palette()
colors = {i: current_palette[i] for i in range(len(centers))}

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

draw = sns.scatterplot(x=0, 
                y=1, 
                data=data,
                hue=clusters_kmeans,
                palette=colors,
                legend=False,
                ax=ax)

for index, row in data.iterrows():
    draw.annotate(str(index), (row[0], row[1]))
                 

plt.show()

Les classes peuvent être identiques, mais l'inertie est beaucoup plus petite pour la deuxième classification.

## Centrer et réduire

On refait l'ACP sur des données centrées et réduite cette fois-ci.

In [ ]:
from sklearn.preprocessing import StandardScaler
import pandas

In [ ]:
scaler = StandardScaler()

X = pandas.DataFrame(scaler.fit_transform(dépense), columns=dépense.columns)

In [ ]:
pca = PCA()
pca.fit(X)

U = np.transpose(pca.components_) # vecteurs propres
I = pandas.DataFrame(np.transpose(pca.explained_variance_ratio_), columns=["pourcentage"])  # information véhiculée

C = pandas.DataFrame(X @ U, index=X.index) # nouvelles coordonnées

In [ ]:
data = C.iloc[:, [0, 1]]

data

In [ ]:
inertie = []

for nb in range(1, 10):
    kmeans = KMeans(n_clusters=nb,
                    n_init=10, 
                    max_iter=300).fit(data)
    inertie.append(kmeans.inertia_)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

sns.lineplot(x=list(range(1, len(inertie) + 1)), 
             y=inertie, 
             legend=False,
             ax=ax)
plt.show()

Ici 4 classes est suffisant et l'inertie est minuscule.

In [ ]:
kmeans = KMeans(n_clusters=4,
                n_init=10, 
                max_iter=300).fit(data)

clusters_kmeans = kmeans.predict(data)

clusters = [[], [], [], []]

for i, cluster in enumerate(clusters_kmeans):
    clusters[cluster].append(dépense.index[i])
    
clusters